In [1]:
#first delete image data from train(cat,dog),,and leave only 800 images on each category(cat and dog),,->train_1000_img
#so we cab apply augmentation and demonstrate accuracy increasing after augmentation

In [2]:
#no need to delete test data,,bcz it is for testing,,not for training,,,lol


# W/O data augmentation,,(ie; on 1600 image cnn is trained...800 each )

In [3]:
import cv2
import os
import random
import numpy as np

In [4]:
mydir=r"D:\DEEP_LEARNING\C_N_N\4_cats_vs_dog_classification\train_800_img"

In [5]:
categories=['cats','dogs']

In [6]:
data=[]

for i in categories:
    folder_path=os.path.join(mydir,i)
    
    if i=='cats':
        label=0
    else:
        label=1
    
    for j in os.listdir(folder_path):
        img_path=os.path.join(folder_path,j)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(150,150))
        data.append([img,label])

In [7]:
random.shuffle(data)

In [8]:
X=[]#X_train
y=[]#y_train
for i in data:
    X.append(i[0])
    y.append(i[1])

In [9]:
y=np.array(y)

In [10]:
X=np.array(X)

In [11]:
X.shape

(1600, 150, 150, 3)

In [12]:
X=X/255

In [16]:
#create CNN
from keras import Sequential
from keras.layers import  Conv2D,MaxPooling2D,Dense,Flatten,Activation,Dropout

In [17]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#convolution layers are finished
model.add(Flatten())#3d feature map to 1d feature map

#F_C layers starts
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [18]:
model.fit(X,y,epochs=5,validation_split=0.1)

Epoch 1/5
45/45 [==============================] - 46s 990ms/step - loss: 0.7142 - accuracy: 0.5278 - val_loss: 0.6945 - val_accuracy: 0.4812
Epoch 2/5
45/45 [==============================] - 41s 918ms/step - loss: 0.7012 - accuracy: 0.5403 - val_loss: 0.6710 - val_accuracy: 0.6062
Epoch 3/5
45/45 [==============================] - 43s 956ms/step - loss: 0.6598 - accuracy: 0.6035 - val_loss: 0.6056 - val_accuracy: 0.7188
Epoch 4/5
45/45 [==============================] - 45s 998ms/step - loss: 0.6390 - accuracy: 0.6493 - val_loss: 0.6449 - val_accuracy: 0.5625
Epoch 5/5
45/45 [==============================] - 45s 998ms/step - loss: 0.6145 - accuracy: 0.6694 - val_loss: 0.5878 - val_accuracy: 0.7125


In [19]:
#as u see  (testing data)val_accuracy: 0.7125 or 71%
#it is on 800 images each for cat and dog
#we can do data augmentation to increase train data sample from current data,,

# applying data augmentation on train_800_img ,,to get more image for training purpose

In [38]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=32

train_datagen=ImageDataGenerator(
                    rescale=1./255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True
                )
test_datagen=ImageDataGenerator(rescale=1./255)

#this is generator that will read images from 
train_generator=train_datagen.flow_from_directory(
                'train_800_img',
                target_size=(150,150),
                batch_size=batch_size,
                class_mode='binary'
                )
validation_generator=test_datagen.flow_from_directory(
        'test_500_img',#this is the target directory
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary') #since we using binary_crossentropy loss , we need binary labels

    

Found 1600 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [39]:
#Found 1600 images belonging to 2 classes.-->train data
#Found 1000 images belonging to 2 classes.-->test data

In [40]:
#exact same model as we above
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#convolution layers are finished
model.add(Flatten())#3d feature map to 1d feature map

#F_C layers starts
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [41]:
#while training the augmented data,,ie; generating and directly feeding to cnn model (ie;not storing),,then use fit_generator(),,to train the model
#but in this current version,u can use fit(...all arguments req to get the augmented data)

In [42]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=800 // batch_size
    )

Epoch 1/25
62/62 [==============================] - 75s 1s/step - loss: 0.7120 - accuracy: 0.5375 - val_loss: 0.6842 - val_accuracy: 0.5675
